## Section A: Image Processing Basics


1.) Load an image using OpenCV and perform the following operations:

• Convert it to grayscale

• Apply Gaussian blur

• Detect edges using Canny edge detection

Importing Libraries and Loading Image 
You need to run this cell at least once for other to work 

In [ ]:
import matplotlib.pyplot as plt
import cv2
print("You are currently using ", cv2.__version__ , " version of cv2")

image=cv2.imread("monster.jpeg")
# cv2 store image in bgr format instead of rgb format so we need to invert it for proper plotting in matplot
cv2.imshow("Image original",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

#i prefer matplotlib for output so i will be converting bgr to rgb for printing
image_mat=image[:,:,::-1]
fig,axes=plt.subplots(1,2)
axes[0].imshow(image)
axes[0].set_title("bgr and rgb are reversed due \n to different format")
axes[1].imshow(image_mat)
axes[1].set_title("Original Image")
plt.axis("off")

Applying grayscale

In [ ]:
gray_scale=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

fig,axes=plt.subplots(1,2)
axes[0].imshow(gray_scale,cmap="gray")
axes[0].set_title("Gray Scaled")
axes[1].imshow(image_mat,cmap="gray")
axes[1].set_title("Original")

Applying Gaussian Blur, you can change the window size by change WINDOW_SIZE using inbuilt Gaussian Blur function 

In [ ]:
WINDOW_SIZE=5
blurred=cv2.GaussianBlur(gray_scale,(WINDOW_SIZE,WINDOW_SIZE),0)

fig,axes=plt.subplots(1,2)
axes[0].imshow(blurred,cmap="gray")
axes[0].set_title("Blurred")
axes[1].imshow(gray_scale,cmap="gray")
axes[1].set_title("Original")

Detecting edges  using inbuilt canny detect function

In [ ]:
edges=cv2.Canny(blurred,100,200)

fig,axes=plt.subplots(1,2)
axes[0].imshow(edges,cmap="gray")
axes[0].set_title("Edges")
axes[1].imshow(blurred,cmap="gray")
axes[1].set_title("Original")

The result can be seen through the following code

In [ ]:
fig,axes=plt.subplots(2,2)
axes[0][0].imshow(image_mat,cmap="gray")
axes[0][0].set_title("Original")
axes[0][1].imshow(gray_scale,cmap="gray")
axes[0][1].set_title("Applied grayscale")
axes[1][0].imshow(blurred,cmap="gray")
axes[1][0].set_title("Applied Gaussian blur")
axes[1][1].imshow(edges,cmap="gray")
axes[1][1].set_title("Applied Canny edge detection")
plt.tight_layout()

2.) Perform image transformations such as rotation, scaling, and flipping using OpenCV. Display the
original and transformed images.

They all have inbuilt functions using them first
You can change the First three variables as per convinience

In [ ]:
SCALING_FACTOR=2
FLIP="Horizontally"
ROTATE="ACW"

Flipcode={
    "Horizontally" : 1 , "Vertically" : 0
}

Rotation={
    "CW": cv2.ROTATE_90_CLOCKWISE,"ACW" : cv2.ROTATE_90_COUNTERCLOCKWISE
}

rotated_90=cv2.rotate(image_mat,Rotation[ROTATE])
scaled=cv2.resize(image_mat,None,fx=SCALING_FACTOR,fy=SCALING_FACTOR)
flipped=cv2.flip(image_mat,Flipcode[FLIP])

fig,axes=plt.subplots(2,2)
axes[0][0].imshow(image_mat,cmap="gray")
axes[0][0].set_title("Original")
axes[0][1].imshow(rotated_90,cmap="gray")
axes[0][1].set_title("Rotated")
axes[1][0].imshow(scaled,cmap="gray")
axes[1][0].set_title("Scaled")
axes[1][1].imshow(flipped,cmap="gray")
plt.tight_layout()

## Section B: Image Classification using CNN


### Implement a CNN for Handwritten Digit Recognition

Import MNIST dataset by running the following cell first:

In [ ]:
import tensorflow.keras as tf
import matplotlib.pyplot as plt
import numpy as np
print(tf.__version__)

(x_train,y_train), (x_test,y_test)=tf.datasets.mnist.load_data()

Training model

In [ ]:
x_train=x_train/255.0
x_test=x_test/255.0

model=tf.Sequential([
    tf.layers.Conv2D(filters=32,kernel_size=(5,5),activation='relu',input_shape=(28,28,1)),
    tf.layers.MaxPooling2D(pool_size=(2,2)),
    tf.layers.Flatten(),
    tf.layers.Dense(10,activation='softmax')
])

model.summary()

Compile and Train Model

In [ ]:
model.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=10, batch_size=1024)

Results:

In [ ]:
test_loss, test_acc = model.evaluate(x_test , y_test, verbose=1)

print('\nTest accuracy:', test_acc)
plt.plot(history.history["loss"], label="Loss")  
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.plot(history.history["accuracy"], label="accuracy")  
plt.xlabel("Epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show()

## Section C: Object Detection using Pre-Trained Models

Kindly import model once by running the following cell

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

model = YOLO("yolov8n.pt") 

You can change or add the paths of the image to be detected by changing images path .( You need to run the follwing cell once to load images)

In [ ]:
images_path=["room.jpeg","monster.jpeg","rainy_night.jpeg"]
images=[cv2.imread(img) for img in images_path]
results=model(images)

By playing the corresponding cell you can see the results

In [ ]:
i=1
plt.figure(figsize=(12,6))
for r in results:
    img=r.plot()
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.subplot(1,len(images_path),i)
    plt.title(images_path[i-1].removesuffix(".jpeg").removesuffix("jpg"))
    plt.imshow(img)
    plt.axis("off")
    i+=1
plt.show()

## Section D: Detect Triangle inside a Circle

1. Use OpenCV API to detect shapes within an image, here triangles and circles.
2. Implement logic to detect whether a triangle lies within another circle, if so, mark(with box or
circle) both the circle and traingle. A triangle may lie within multiple circles.
3. Submit atleast 3 images(testcases) validating your implementation.

### Part A: detecting circle using Hough circle method

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
print(cv2.__version__)
img=cv2.imread("circles.jpeg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
blurred=cv2.blur(gray,(3,3))
canny=cv2.Canny(blurred,10,250)
plt.imshow(canny,cmap="gray")

In [ ]:
detected_circles = cv2.HoughCircles(canny,cv2.HOUGH_GRADIENT,1,20,param1=30,param2=30,minRadius=50,maxRadius=100)
if detected_circles is not None: 
    detected_circles=np.uint16(np.around(detected_circles))
    for pt in detected_circles[0,:]: 
        a, b, r = pt[0], pt[1], pt[2] 
        cv2.circle(img, (a, b), r, (255, 255, 0), 2) 
        cv2.circle(img, (a, b), 1, (0, 0, 255), 3)

plotable_img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) 
plt.imshow(plotable_img)
plt.axis("off")

### Part B: detecting line using Hough line detection method

converting image in to usable format

In [ ]:
img=cv2.imread("triangles.jpeg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blurred=cv2.blur(gray,(3,3))
canny=cv2.Canny(blurred,20,200)

plt.imshow(canny,cmap="gray")

Detecting lines using inbuilt functions:

In [ ]:
lines=cv2.HoughLinesP(canny,1,np.pi/180,50,minLineLength=2,maxLineGap=1)
lines_list=list()
for line in lines:
    x1,y1,x2,y2=line[0]
    cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)
    lines_list.append([(x1,y1),(x2,y2)])

plotable_img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(plotable_img)
plt.axis("off")

Triangle detection
(Method 1 : O(n<sup>3</sup>) approach)

In [ ]:
def is_triangle(p1, p2, p3):
    return (p1[0] * (p2[1] - p3[1]) +
            p2[0] * (p3[1] - p1[1]) +
            p3[0] * (p1[1] - p2[1])) != 0

for i in range(len(lines_list)):
    first_line = lines_list[i]

    for j in range(i + 1, len(lines_list)):
        second_line = lines_list[j]

        common_point = set(first_line) & set(second_line)
        if len(common_point) != 1:
            continue
        
        common_point = list(common_point)[0]
        remaining_points = set(first_line + second_line) - {common_point}

        for k in range(j + 1, len(lines_list)):
            third_line = lines_list[k]

            if set(third_line) == remaining_points:
                p1, p2, p3 = first_line[0], first_line[1], second_line[1]

                if is_triangle(p1, p2, p3):
                    cv2.line(img, p1, p2, (0, 0, 255), 3)
                    cv2.line(img, p2, p3, (0, 0, 255), 3)
                    cv2.line(img, p3, p1, (0, 0, 255), 3)

plotable_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(plotable_img)
plt.axis("off")
plt.show()

Found Out that above approach has some flaws since the end point are not exactly overlapping in detection so using other methods  

### Triangle detection using contour curve

In [ ]:
img=cv2.imread("triangles.jpeg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur=cv2.blur(img,(3,3))
canny=cv2.Canny(blur,10,250)

contours , heirarchy = cv2.findContours(canny, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_NONE)

for contour in contours:

    epsilon=0.05 * cv2.arcLength(contour,True)
    approx_poly=cv2.approxPolyDP(contour,epsilon,True)

    if len(approx_poly)==3:
        cv2.drawContours(img,[approx_poly],0,(0,0,255),3)

plotable_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(plotable_img)
plt.axis("off")
plt.show()

    Detecting triangle in a circle using contour curve ( combining all above methods)

In [ ]:
#converting image
img=cv2.imread("Screenshot 2025-03-28 123548.png")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur=cv2.GaussianBlur(gray,(9,9),2)
canny=cv2.Canny(blur,50,200)

#finding circles
detected_circles = cv2.HoughCircles(canny,cv2.HOUGH_GRADIENT,1,20,param1=120,param2=30,minRadius=20,maxRadius=200)
if detected_circles is not None: 
    detected_circles=np.uint16(np.around(detected_circles))

#finding triangles
contours , heirarchy = cv2.findContours(gray, cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    epsilon=0.05 * cv2.arcLength(contour,True)
    approx_poly=cv2.approxPolyDP(contour,epsilon,True)

    if len(approx_poly)==3:
        #checking if trinagle is in any circle
        for circle in detected_circles[0,:]:
            a,b,r=circle[0],circle[1],circle[2]
            if all((pt[0]-a)**2 + (pt[1]-b)**2 <= r**2 for pt in approx_poly[:,0,:]):
                #triangle detected inside a circle
                cx,cy,re=cv2.minEnclosingCircle(approx_poly)
                cv2.circle(img,(int(cx),int(cy)),int(re),(0,255,0),3)
                cv2.circle(img,(a,b),r,(255,0,0),3)


plotable_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(plotable_img)
plt.axis("off")
plt.show()

